In [1]:
import Pkg; Pkg.add("DelimitedFiles"); using DelimitedFiles;

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
 Resolving package versions...
  Updating `~/.julia/environments/v1.1/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.1/Manifest.toml`
 [no changes]


In [2]:
function tokenize(str)
    out = []
    str = split(str)
    symbols = ['(', ':',';',',', ')', '.', '?', '!']
    finish_symbols = [')', ',', '.']
    bad_strings = ["<br", "/><br", "/>", "/><br", ".<br", "\"", "..."]
    for word in str 
        f = true
        word = lowercase(word)
        put_symbols = []
        for symbol in symbols
            if occursin(string(symbol), word); word = strip(word, [symbol]); push!(put_symbols, symbol); f=false; end;
        end
        for bad in bad_strings
            if occursin(bad, word); word = replace(word, bad=>""); f=false; break; end
        end
        for s in put_symbols
            if !(s in finish_symbols); push!(out, s); end
        end
        push!(out, word)
        for s in put_symbols
            if (s in finish_symbols); push!(out, s); end
        end
    end
    #remove zero-length strings
    out = [word for word in out if length(word) > 0]
end

tokenize (generic function with 1 method)

In [3]:
train_pos_path = "/Users/egeersu/Desktop/NLP/hw1/aclImdb/train/pos"
train_pos_files = readdir(train_pos_path)
train_x_pos = []

for file in train_pos_files
    file_path = train_pos_path * '/' * file
    io = open(file_path, "r")
    review = readlines(io)[1]
    push!(train_x_pos, review)
    close(io)
end

train_x_pos = tokenize.(train_x_pos)
train_pos = [(t,1) for t in train_x_pos]
length(train_pos)

12500

In [4]:
train_neg_path = "/Users/egeersu/Desktop/NLP/hw1/aclImdb/train/neg"
train_neg_files = readdir(train_neg_path)
train_x_neg = []

for file in train_neg_files
    file_path = train_neg_path * '/' * file
    io = open(file_path, "r")
    review = readlines(io)[1]
    push!(train_x_neg, review)
    close(io)
end

train_x_neg = tokenize.(train_x_neg)
train_neg = [(t,1) for t in train_x_neg]
length(train_neg)

12500

In [5]:
test_pos_path = "/Users/egeersu/Desktop/NLP/hw1/aclImdb/test/pos"
test_pos_files = readdir(test_pos_path)
test_x_pos = []

for file in test_pos_files
    file_path = test_pos_path * '/' * file
    io = open(file_path, "r")
    review = readlines(io)[1]
    push!(test_x_pos, review)
    close(io)
end

test_x_pos = tokenize.(test_x_pos)
test_pos = [(t,1) for t in test_x_pos]
length(test_pos)

12500

In [6]:
test_neg_path = "/Users/egeersu/Desktop/NLP/hw1/aclImdb/test/neg"
test_neg_files = readdir(test_neg_path)
test_x_neg = []

for file in test_neg_files
    file_path = test_neg_path * '/' * file
    io = open(file_path, "r")
    review = readlines(io)[1]
    push!(test_x_neg, review)
    close(io)
end

test_x_neg = tokenize.(test_x_neg)
test_neg = [(t,1) for t in test_x_neg]
length(test_neg)

12500

**Positive Count**

In [7]:
#count the number of times each word appears in train_data
pos_counter = Dict()
for t in train_pos
    for word in t[1]
        if word in keys(pos_counter); pos_counter[word] += 1; else; pos_counter[word] = 1; end
    end
end

total_pos = 0
for k in keys(pos_counter); total_pos += pos_counter[k]; end
"Total number of words: ", total_pos

("Total number of words: ", 3303288)

In [8]:
for (k,v) in pos_counter 
    if v < 2; delete!(pos_counter, k); end
end

**Negative Count**

In [9]:
#count the number of times each word appears in train_data
neg_counter = Dict()
for t in train_neg
    for word in t[1]
        if word in keys(neg_counter); neg_counter[word] += 1; else; neg_counter[word] = 1; end
    end
end

total_neg = 0
for k in keys(neg_counter); total_neg += neg_counter[k]; end
"Total number of words: ", total_neg

("Total number of words: ", 3222796)

In [10]:
for (k,v) in neg_counter 
    if v < 2; delete!(neg_counter, k); end
end

**Naive Bayes Model**

In [11]:
function predict_positive(x)
    product = log(0.5)
    for token in x; product += log(lookup_count(token, pos_counter) / total_pos); end
    product
end

function predict_negative(x)
    product = log(0.5)
    for token in x; product += log(lookup_count(token, neg_counter) / total_neg); end
    product
end

function predict(x)
    if predict_negative(x) < predict_positive(x); "positive"; else; "negative"; end
end

predict (generic function with 1 method)

In [12]:
function lookup_count(token, counter); if token in keys(counter); return counter[token]; else; return 1; end; end

lookup_count (generic function with 1 method)

**Accuracy**

In [13]:
#Train Accuracy
total = 0
correct = 0
for x in train_x_neg
    total += 1
    if predict(x) == "negative"; correct += 1; end
end
for x in train_x_pos
    total += 1
    if predict(x) == "positive"; correct += 1; end
end

accuracy_train = correct/total

0.89608

In [14]:
#Test Accuracy
total = 0
correct = 0
for x in test_x_neg
    total += 1
    if predict(x) == "negative"; correct += 1; end
end
for x in test_x_pos
    total += 1
    if predict(x) == "positive"; correct += 1; end
end

accuracy_test = correct / total

0.81388